## This notebook shows how to save data

In [ ]:
import lzma
import pathlib
import pickle
import tempfile
import time

import laueimproc

### Let initialise somme objects

In [ ]:
temp_dir = pathlib.Path(tempfile.gettempdir())
dataset = laueimproc.DiagramsDataset(laueimproc.io.get_samples())

In [ ]:
def peaks_search(diagram: laueimproc.Diagram):
    """Basic peaks search."""
    diagram.find_spots()
    
def compress(diagram: laueimproc.Diagram):
    """Keep the 4 smallest first spots."""
    diagram.filter_spots(
        (diagram.bboxes[:, 2]*diagram.bboxes[:, 3]).argsort()[:4],
        "keeps 4 spots"
    )

_ = dataset.apply(peaks_search)
# _ = dataset.apply(compress)

In [ ]:
print(dataset)

### Basic Manual serialization on diagrams

In [ ]:
# serialize diagram
diagram = dataset[0]
data = pickle.dumps(diagram)
data = lzma.compress(data)
print(len(data))

with open(temp_dir / "diagram.pickle.xz", "wb") as raw:
    raw.write(data)

In [ ]:
# deserialize a diagram
with open(temp_dir / "diagram.pickle.xz", "rb") as raw:
    data = raw.read()
data = lzma.decompress(data)
diagram_bis = pickle.loads(data)

print(diagram)
print()
print(diagram_bis)

### Basic Manual serialization on dataset

In [ ]:
# serialize with pickle
with open(temp_dir / "dataset.pickle.xz", "wb") as raw:
    pickle.dump(dataset, raw)

In [ ]:
# deserialize with pickle
with open(temp_dir / "dataset.pickle.xz", "rb") as raw:
    dataset_bis = pickle.load(raw)

print(dataset)
print()
print(dataset_bis)

### Save dataset from interface (checkpoint)

In [ ]:
filename = temp_dir / "dataset.pickle"

In [ ]:
# create a new dataset from backup
new_dataset = laueimproc.DiagramsDataset()
new_dataset.restore(filename)
print(new_dataset)

In [ ]:
# let see the underground activity
dataset.autosave(filename, delay="30s")
for _ in range(10):
    print(f"last write time: {filename.stat().st_mtime}")
    time.sleep(6)